## Obtencion del factor temporal de los códigos

In [2]:
from PyPDF2 import PdfReader, PdfWriter

def split_pdf(input_pdf, start_page, end_page, output_pdf):
    reader = PdfReader(input_pdf)
    writer = PdfWriter()

    for i in range(start_page - 1, end_page):
        writer.add_page(reader.pages[i])

    with open(output_pdf, "wb") as out_file:
        writer.write(out_file)

# Ejemplo de uso
split_pdf("../../data\Documentación ficheros de usuario EPF 2006 Año 2021.pdf", 64, 259, "../../data/anexo_codes.pdf")


In [3]:
import pdfplumber
import re

def extract_codes_and_factors(pdf_path):
    keywords = {"Bisemanal", "Mensual", "Trimestral", "Último", "Anual"}  # Factores posibles
    data = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                lines = text.split("\n")  # Dividir en líneas
                
                for i in range(len(lines) - 1):
                    line = lines[i].strip()  # Eliminar espacios y tabulaciones al inicio

                    # Buscar código al inicio de la línea, permitiendo que el último carácter sea una letra
                    match = re.match(r'^(\d{2})\.(\d)\.(\d)\.([\dA-Z])', line)
                    if match:
                        clean_code = "".join(match.groups())  # Unir los números y letras sin puntos

                        # Buscar el factor en las siguientes 6 líneas
                        factor = "NaN"
                        for j in range(1, 7):  # Buscar hasta 6 líneas más abajo
                            if i + j < len(lines):
                                next_line = lines[i + j].strip()  # Eliminar espacios en la siguiente línea
                                first_word = next_line.split()[0] if next_line else ""  # Tomar la primera palabra
                                
                                if first_word in keywords:
                                    factor = first_word
                                    break  # Si ya encontramos un factor, salimos del bucle

                        data.append((clean_code, factor))

    return data


In [4]:
import pandas as pd

# 🔹 Extraer códigos y factores del PDF
pdf_path = "../../data/anexo_codes.pdf"  # Reemplázalo con la ruta correcta
codes_factors = extract_codes_and_factors(pdf_path)

# 🔹 Crear DataFrame y asegurar que `CODIGO` sea string
df = pd.DataFrame(codes_factors, columns=["CODIGO", "FACTOR_TEMPORAL"])
df["CODIGO"] = df["CODIGO"].astype(str)  # Convertir a string por seguridad



In [5]:
for code, factor in codes_factors:
    print(f"{code} -> {factor}")

01111 -> Bisemanal
01112 -> Bisemanal
01113 -> Bisemanal
01114 -> Bisemanal
01115 -> Bisemanal
01116 -> Bisemanal
01117 -> Bisemanal
01118 -> Bisemanal
01121 -> Bisemanal
01122 -> Bisemanal
01123 -> Bisemanal
01124 -> Bisemanal
01125 -> Bisemanal
01126 -> Bisemanal
01127 -> Bisemanal
01128 -> Bisemanal
01131 -> Bisemanal
01132 -> Bisemanal
01133 -> Bisemanal
01134 -> Bisemanal
01135 -> Bisemanal
01136 -> Bisemanal
01141 -> Bisemanal
01142 -> Bisemanal
01143 -> Bisemanal
01144 -> Bisemanal
01145 -> Bisemanal
01146 -> Bisemanal
01147 -> Bisemanal
01151 -> Bisemanal
01152 -> Bisemanal
01153 -> Bisemanal
01154 -> Bisemanal
01155 -> Bisemanal
01161 -> Bisemanal
01162 -> Bisemanal
01163 -> Bisemanal
01164 -> Bisemanal
01165 -> Bisemanal
01166 -> Bisemanal
01167 -> Bisemanal
01168 -> Bisemanal
01169 -> Bisemanal
01160 -> Bisemanal
01171 -> Bisemanal
01172 -> Bisemanal
01173 -> Bisemanal
01174 -> Bisemanal
01175 -> Bisemanal
01176 -> Bisemanal
01177 -> Bisemanal
01178 -> Bisemanal
01179 -> Bis

Manejamos manualmente los NaN

In [6]:
# Quiero ver de df las filas que FACTOR_TEMPORAL sea igual a Nan
print(df[df["FACTOR_TEMPORAL"] == "NaN"])

    CODIGO FACTOR_TEMPORAL
105  04210             NaN
106  04221             NaN
107  04222             NaN
108  04223             NaN
181  04122             NaN
194  06212             NaN
196  06212             NaN
214  04124             NaN
294  10400             NaN
295  10222             NaN
302  10214             NaN
360  09210             NaN


In [7]:
df.loc[df["CODIGO"] == "09210", "FACTOR_TEMPORAL"] = "Anual"
df.loc[df["CODIGO"].astype(str).str.startswith("0422"), "FACTOR_TEMPORAL"] = "Último"
df.loc[df["CODIGO"] == "06212", "FACTOR_TEMPORAL"] = "Trimestral"
df.loc[df["CODIGO"].isin(["10214", "10400", "04124", "04210", "04122", "10222"]), "FACTOR_TEMPORAL"] = "Último"



In [8]:
df = df.drop_duplicates(subset=["CODIGO"])

In [9]:
df[df["FACTOR_TEMPORAL"] == "NaN"]

,CODIGO,FACTOR_TEMPORAL


Comparamos con los codigos de gastos para ver que códigos nos faltan

In [10]:
df_gastos = pd.read_csv("../../DataLake/2023/family_expenses.tsv", sep="\t", encoding="utf-8")

In [11]:
codigos_unicos = df_gastos[~df_gastos["CODIGO"].isin(df["CODIGO"])]
codigos_unicos["CODIGO"].unique()

array(['0117B', '0117A', '0117C'], dtype=object)

In [12]:
nuevos_codigos = pd.DataFrame({
    "CODIGO": ['0117A', '0117B', '0117C'],
    "FACTOR_TEMPORAL": ["Bisemanal"] * 3
})

# Concatenar con el DataFrame existente
df = pd.concat([df, nuevos_codigos], ignore_index=True)


In [13]:
codigos_unicos = df_gastos[~df_gastos["CODIGO"].isin(df["CODIGO"])]
codigos_unicos["CODIGO"].unique()

array([], dtype=object)

Guardamos los códigos con sus factores

In [16]:
# 🔹 Guardar en CSV con separación por tabuladores
df.to_csv("../../data/codes_factors.csv", sep="\t", index=False)

### Creamos un datamart con los gastos agregados por super categorías

Primero cargamos el datalake

In [17]:
import os
import glob
import pandas as pd

In [18]:
def load_datalake(datalake_path):
    """
    Carga los archivos family_expenses.tsv de todos los años dentro del datalake y los concatena en un único DataFrame.
    Añade una columna 'Año' correspondiente al año de cada archivo.
    """
    all_files = glob.glob(os.path.join(datalake_path, "*", "family_expenses.tsv"))
    df_list = []

    for file in all_files:
        year = os.path.basename(os.path.dirname(file))  # Extraer el año de la ruta
        df = pd.read_csv(file, sep="\t")  # Cargar el archivo
        df["Año"] = int(year)  # Añadir la columna Año
        df_list.append(df)

    if not df_list:
        raise ValueError("No se encontraron archivos family_expenses.tsv en la ruta del datalake.")

    # Concatenar todos los DataFrames
    df_expenses = pd.concat(df_list, ignore_index=True)
    return df_expenses


In [19]:

def normalize_expenses(df):
    """
    Normaliza la columna GASTO dividiendo por FACTOR y ajustándola a nivel mensual según FACTOR_TEMPORAL.
    """
    # Definir los días asociados a cada factor temporal
    dias_por_periodo = {
        "Bisemanal": 14,
        "Mensual": 30,
        "Trimestral": 90,
        "Anual": 365,
        "Último": 30  # Último recibo ya se considera mensual
    }

    # Convertir FACTOR_TEMPORAL a días
    df["DIAS_PERIODO"] = df["FACTOR_TEMPORAL"].map(dias_por_periodo)

    # Normalizar GASTO
    df["GASTO_HOGAR"] = df["GASTO"] / df["FACTOR"]
    df["GASTO_MENSUAL"] = df["GASTO_HOGAR"] / 12

    return df

In [20]:
def aggregate_supercategories_two_digits(df):
    """
    Agrega los datos por supercategorías, utilizando los dos primeros dígitos del código como categoría.
    Agrupa por AÑO, NUMERO y SUPER_CATEGORIA.
    """
    df["SUPER_CATEGORIA"] = df["CODIGO"].astype(str).str[:2]  # Extraer los dos primeros dígitos del código

    # Agregar los gastos por supercategoría, año y número de hogar
    df_agg = df.groupby(["Año", "NUMERO", "SUPER_CATEGORIA"])["GASTO_MENSUAL"].sum().reset_index()

    return df_agg

In [21]:
# Ruta del Datalake
datalake_path = "../../Datalake/"  # Ajusta según la ubicación real

# Cargar los datos del datalake
df_expenses = load_datalake(datalake_path)

# Normalizar los gastos
df_expenses = normalize_expenses(df_expenses)

# Agregar por supercategorías
df_aggregated = aggregate_supercategories_two_digits(df_expenses)

# Mostrar el DataFrame con los datos agregados
df_aggregated.head()

,Año,NUMERO,SUPER_CATEGORIA,GASTO_MENSUAL
0,2016,1,01,88.712448
1,2016,1,02,2.970634
2,2016,1,03,18.523350
3,2016,1,04,455.425600
4,2016,1,05,4.436967


In [ ]:
# crear directorio de salida si no existe
output_dir = "../../datamarts"
os.makedirs(output_dir, exist_ok=True)

In [23]:
# Guardar el DataFrame agregado como un archivo TSV
output_filename = "../../datamarts/datamart_supercategories.tsv"
df_aggregated.to_csv(output_filename, sep="\t", index=False)

print(f"Datamart saved as: {output_filename}")


Datamart saved as: ../../datamarts/datamart_supercategories.tsv


In [24]:
def aggregate_supercategories_three_digits(df):
    """
    Agrega los datos por supercategorías, utilizando los dos primeros dígitos del código como categoría.
    Agrupa por AÑO, NUMERO y SUPER_CATEGORIA.
    """
    df["SUPER_CATEGORIA"] = df["CODIGO"].astype(str).str[:3]  # Extraer los dos primeros dígitos del código

    # Agregar los gastos por supercategoría, año y número de hogar
    df_agg = df.groupby(["Año", "NUMERO", "SUPER_CATEGORIA"])["GASTO_MENSUAL"].sum().reset_index()

    return df_agg

In [ ]:
# Cargar los datos del datalake
df_expenses = load_datalake(datalake_path)

# Normalizar los gastos
df_expenses = normalize_expenses(df_expenses)

# Agregar por supercategorías
df_aggregated = aggregate_supercategories_three_digits(df_expenses)

# Mostrar el DataFrame con los datos agregados
df_aggregated.head()

In [ ]:
# Guardar el DataFrame agregado como un archivo TSV
output_filename = "datamarts/datamart_supercategories_three_digits.tsv"
df_aggregated.to_csv(output_filename, sep="\t", index=False)

print(f"Datamart saved as: {output_filename}")


Datamart saved as: datamarts/datamart_supercategories_three_digits.tsv


### Gastos porcentuales por super categoría y casa

In [ ]:
import pandas as pd
from pathlib import Path

def leer_datalake(path_datalake: str) -> pd.DataFrame:
    """Lee todos los archivos homes.tsv del datalake y añade la columna 'Año'."""
    datalake_path = Path(path_datalake)
    homes_dfs = []

    for year_dir in datalake_path.iterdir():
        if year_dir.is_dir():
            homes_file = year_dir / "homes.tsv"
            if homes_file.exists():
                df = pd.read_csv(homes_file, sep="\t", usecols=["NUMERO", "IMPEXAC"])
                df["Año"] = int(year_dir.name)
                homes_dfs.append(df)

    if not homes_dfs:
        raise ValueError("No se encontraron archivos homes.tsv en el datalake.")

    return pd.concat(homes_dfs, ignore_index=True)

def leer_datamart(path_datamart: str) -> pd.DataFrame:
    """Lee el datamart de supercategorías con gasto mensual."""
    return pd.read_csv(path_datamart, sep="\t", usecols=["Año", "NUMERO", "SUPER_CATEGORIA", "GASTO_MENSUAL"])

def calcular_porcentaje_gasto(df: pd.DataFrame) -> pd.DataFrame:
    """Calcula el porcentaje del gasto por supercategoría respecto al gasto total del hogar."""
    total_gasto = df.groupby(["Año", "NUMERO"])["GASTO_MENSUAL"].transform("sum")
    df["porcentaje_gasto"] = (df["GASTO_MENSUAL"] / total_gasto) * 100
    df["porcentaje_gasto"] = df["porcentaje_gasto"].round(2)
    return df

def procesar_datalake_y_datamart(path_datalake: str, path_datamart: str) -> pd.DataFrame:
    """Pipeline completo para unir datamart con ingresos del datalake y calcular porcentaje de gasto."""
    homes_df = leer_datalake(path_datalake)
    datamart_df = leer_datamart(path_datamart)

    # Calcular porcentaje de gasto sobre el total de gastos del hogar
    datamart_df = calcular_porcentaje_gasto(datamart_df)

    # Añadir IMPEXAC al dataset final
    merged_df = pd.merge(datamart_df, homes_df, on=["Año", "NUMERO"], how="left")

    # Seleccionar columnas finales
    return merged_df[["Año", "NUMERO", "SUPER_CATEGORIA", "porcentaje_gasto", "IMPEXAC"]]


In [ ]:
df_resultado = procesar_datalake_y_datamart(datalake_path, "../../datamarts/datamart_supercategories.tsv")
print(df_resultado.head())


    Año  NUMERO  SUPER_CATEGORIA  porcentaje_gasto  IMPEXAC
0  2016       1                1              5.55     1225
1  2016       1                2              0.19     1225
2  2016       1                3              1.16     1225
3  2016       1                4             28.52     1225
4  2016       1                5              0.28     1225


In [ ]:
# 🔹 Guardar el resultado en un archivo TSV
output_filename = "../../datamarts/porcentaje_gasto_supercategorias.tsv"
df_resultado.to_csv(output_filename, sep="\t", index=False)
print(f"Archivo guardado: {output_filename}")

Archivo guardado: datamarts/porcentaje_gasto_supercategorias.tsv
